In [2]:
import ray
import pandas as pd
import time 
import numpy as np
import binascii
import redis
import pprint
pp = pprint.PrettyPrinter() # for printing dicts and lists in a manner easy for the eyes
from misc import *


In [3]:
ray.init(num_cpus=3)

Waiting for redis server at 127.0.0.1:47883 to respond...
Waiting for redis server at 127.0.0.1:15885 to respond...
Starting local scheduler with 3 CPUs and 0 GPUs.


{'local_scheduler_socket_names': ['/tmp/scheduler42909302'],
 'node_ip_address': '127.0.0.1',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/plasma_store63297133', manager_name='/tmp/plasma_manager45575887', manager_port=53037)],
 'redis_address': '127.0.0.1:47883'}

### Below, we define some example functions for populating the redis database

In [ ]:
# Function definitions...

@ray.remote
def example(x):
    time.sleep(np.random.random())
    return np.random.randn()

@ray.remote
class TestCls():
    def __init__(self):
        self.g = 1
        
    def to_go(self, x):
        return x

    
@ray.remote
class Outer():
    def __init__(self):
        self.f = 1
        self.test = TestCls.remote()
    
    def to_go2(self, x):
        return x * 2
    
    def error(self):
        return 1/0

In [ ]:
# Here, we generate data in redis for remote tasks
results = ray.get([example.remote(x) for x in range(4)])

# Generating data for Actor tasks
actor = TestCls.remote()
actor_results = ray.get([actor.to_go.remote(1)])

err_actor = Outer.remote()
err_actor.error.remote()

In [7]:
global_state = ray.global_state # Used to access redis client, but also has nice interface for certain information 
addr, port = ray.worker.global_worker.redis_address.split(":")
rc = redis.StrictRedis(host=addr, port=port, decode_responses=True)
# rc_non = global_state.redis_client # Redis Client for interacting with redis without decoding

From checking the values of the redis store, we can identify some key things included via `rc.keys()`:
 - Event log
 - worker info
 - Remote functions
 - "Functions to run"
 - Drivers
 - Redis clients
 - Actor classes
 - Actors

In [ ]:
# for k in rc.keys():
#     try:
#         print(k)
#         pp.pprint(rc.hgetall(k))
# #         rc.hgetall(k) 
#     except Exception:
#         print(k, "Failed ")
# # Certain return values, such as `event_log:*`, can only be accessed via list calls to redis

#         try:
#             print('#' * 10)
#             print(k)
#             pp.pprint(rc.lrange(k, 0, -1))
#         except Exception:
#             print(k, "Failed ")
            

## Actor Info


# TODO: Fix actor_id

In [ ]:
# We can populate a DataFrame with Actor 
actor_list = []
for a_key in rc.keys("Actor:*"):
    v = rc.hgetall(a_key)
    v['actor_id'] = a_key
    v['class_id'] = hex_identifier(v['class_id'])
    actor_list.append(v)

actor_df = pd.DataFrame(actor_list)

In [ ]:
actor_df

## Actor Classes Info

In [ ]:
actor_classes = []
for a_key in rc.keys("ActorClass:*"):
    
    v = rc.hgetall(a_key)
    del v['class'] # removed pickle hex for readability
    v['driver_id'] = hex_identifier(v['driver_id'])
    class_id = a_key.split(b':')[1]
    v['class_id'] = hex_identifier(class_id)
    actor_classes.append(v)
    
actor_class_df = pd.DataFrame(actor_classes)

In [ ]:
actor_class_df

## Remote Functions Info

In [ ]:
fn_table = global_state.function_table()
fn_list = []
for fn_id in fn_table:
    val = fn_table[fn_id]
    val["function_id"] = fn_id
    fn_list.append(val)
pd.DataFrame(fn_list)

## Node Info

In [8]:
# Using the global state API, we can populate a DataFrame with a list of Redis Clients currently connected
ctable = global_state.client_table()

client_list = []
for node_ip in ctable:
    for client in ctable[node_ip]:
        client["node_ip_address"] = node_ip
        client_list.append(client)

client_df = pd.DataFrame(client_list)
client_df

,AuxAddress,ClientType,DBClientID,Deleted,LocalSchedulerSocketName,NumCPUs,NumGPUs,node_ip_address
0,NaN,plasma_manager,0d0de2a4834950c2300fc56c30a35f3aaebf82e1,False,NaN,NaN,NaN,127.0.0.1
1,127.0.0.1:53037,local_scheduler,29b3e3c39c6318351fda75dac0c9d2405df190e0,False,/tmp/scheduler42909302,3.0,0.0,127.0.0.1
2,NaN,global_scheduler,7ecef464394486465ec9a0e0c4e82fbae36b9c1d,False,NaN,NaN,NaN,127.0.0.1


In [ ]:
# We can populate a DataFrame with a list of objects in the object store
object_dict = {oid.hex(): v for oid, v in global_state.object_table().items()}
object_df = pd.DataFrame(object_dict).transpose()
object_df
# May need to change this so ID is not index

## Task Info

In [ ]:
from pandas.io.json import json_normalize

tt = global_state.task_table()
tt_list = list(tt.values())

for d in tt_list:
    d['TaskSpec']['ReturnObjectIDs'] = [oid.hex() for oid in d['TaskSpec']['ReturnObjectIDs']]

task_df = json_normalize(tt_list)

task_df

We can also parse the event logs in order to get fine grained timing for remote tasks. However, as a user, I'd probably only care about time taken in running the task -- this can be much refined.

In [ ]:
import json 
event_list = []

# Get and decode all task timing/event logs
for key in rc.keys("event_log*"):
    content = rc.lrange(key, 0, -1)
#     event_list.append(json.loads(content[0].decode())) 
    event_list.append(json.loads(content[0]))
    
# event_list

In [ ]:
from collections import defaultdict

# This seems to be the best way to do the event_log -> dataframe pipeline. 
# First generate a (key, [value]) mapping for all tasks and then apply some pandas operations to convert.

# event_dict is used to store timing info
event_dict = defaultdict(lambda: np.full(len(event_list), np.nan))

# info_dict is used to store meta data - such as function names and task id
info_dict = defaultdict(lambda: [None] * len(event_list))

for i, task_event in enumerate(event_list):
    for event in (task_event):
        time, label, startstop, info = event
        event_dict[(label, startstop)][i] = time
        if info:
            for k in info:
                info_dict[k][i] = info[k]

## Event Timing

In [ ]:
# The tuple keys for `event_dict` produce a hierarchical index, which could be useful. However, joining or merging it 
# with other non-hierarchical indices will throw away this structure.

edf = pd.DataFrame(event_dict) 
edf.rename(columns={1: 'start', 2:'end'}, inplace=True)
edf

## Event Info

In [ ]:
idf = pd.DataFrame(info_dict)
# idf.columns = pd.MultiIndex.from_tuples([(c, '') for c in idf]) # this is non-idempotent!
idf

In [ ]:
remote_table = pd.concat([idf, edf], axis=1)

In [ ]:
# Example to get Error, function_name, IP address, Actor ID, ParentID

errors = idf[idf.traceback.notnull()]
error_task = errors.merge(task_df, left_on="task_id", right_on="TaskSpec.TaskID")
err_task_loc = error_task.merge(client_df, left_on="LocalSchedulerID", right_on="DBClientID")
err_task_loc[['function_name', 'traceback', 'value', 'TaskSpec.ActorID',
              'TaskSpec.ParentTaskID',  'node_ip_address',  'task_id',]]

# TODO:

- Error messages logging (specifics - which node, which function call, which actor, what time)
  - Getting Error messages from Redis is redundant because we already get info in the event_log. Error messages provide extra information such as `error_id` and `type`, which don't seem particularly useful.
  - ~Create table for workers (Id, socket info, node IP address) ... are workers even still a proper abstraction~
  - Get multinode setting - test out client table
  - !! Write out an example for error tracing